In [1]:
import numpy as np
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics.IndependentCascadesModel as ids
import ndlib.models.epidemics.ThresholdModel as th
from tqdm import tqdm
import IPython.display as ipd


def intt(x):
    return int(float(x))

def buildGraph(adjList,numOfNodes):
    numOfEle = np.shape(adjList)[0]
    g = nx.empty_graph(numOfNodes-1)
    for i in range (0,numOfEle):
        g.add_edge(adjList[i,0],adjList[i,1])
    return (g)


### read network topology

In [2]:
# Type of IM seed
IMmethod = "Betweenness Centrality"
# set initial values
numOfSims = 1
numOfIters = 30
# Network topology
dataset = "CA-HepTh-2"
numOfNodes = 9877


# read the graphs from file
augmentedAdj = np.genfromtxt("Data/" + dataset + "/10 IM ready Graphs with Diffusion Probabilities/Cenrality/augmented.csv")
originalAdj = np.genfromtxt("Data/" + dataset + "/10 IM ready Graphs with Diffusion Probabilities/Cenrality/original.csv")
randomAdj = np.genfromtxt("Data/" + dataset + "/10 IM ready Graphs with Diffusion Probabilities/Cenrality/random.csv")

# build the graphs
augmentedGraph = buildGraph(augmentedAdj,numOfNodes)
originalGraph = buildGraph(originalAdj,numOfNodes)
randomGraph = buildGraph(randomAdj,numOfNodes)

# read the seed set from the file
augmentedSeed = np.genfromtxt("Data/" + dataset + "/11 IM stage results/" + IMmethod + "/augmented.csv",dtype=int)
originalSeed = np.genfromtxt("Data/" + dataset + "/11 IM stage results/" + IMmethod + "/original.csv",dtype=int)
randomSeed = np.genfromtxt("Data/" + dataset + "/11 IM stage results/" + IMmethod + "/random.csv",dtype=int)

### simulation paprameters

In [3]:
# g: input graph
# seed: input seed
def simulation(g,adjList,seed):
    numOfEle = np.shape(adjList)[0]
    # an array to keep track of the number of infected at each iteration
    infectedArray = np.zeros(numOfIters)

    # arrays to keep data of each simulations
    arrOfSus = []
    arrOfInfAndRem = []
    
    for i in tqdm(range(0,numOfSims)):
        # Model selection
        model = ids.IndependentCascadesModel(g)
        # Model Configuration
        config = mc.Configuration()
        # config.add_model_parameter
        config.add_model_initial_configuration("Infected", seed)
        # Setting the edge parameters
        for j in range (0,numOfEle):
            config.add_edge_configuration("threshold",(adjList[i,0],adjList[i,1]) , adjList[i,2])
        model.set_initial_status(config)

        # Simulation execution
        for j in range(0,numOfIters):
            iterations = model.iteration_bunch(1)
            trends = model.build_trends(iterations)
            numOfInfected = sum(value == 1 for value in model.status.values()) + sum(value == 2 for value in model.status.values())
            infectedArray[j] = (infectedArray[j] * i + numOfInfected)/(i+1)
    di = model.status
    infected = np.array(list(filter(lambda x: di[x] in [1, 2], di)))
    return(infected)
#         print('------------------------------------')
#         print(sum(value == 0 for value in model.status.values()))
#         print(sum(value == 1 for value in model.status.values()))
#         print(sum(value == 2 for value in model.status.values()))
        


### run simulation per graphs

In [5]:
augmentedInfected = simulation(augmentedGraph,augmentedAdj,augmentedSeed)
originalInfected = simulation(originalGraph,originalAdj,originalSeed)
randomInfected = simulation(randomGraph,randomAdj,randomSeed)

100%|██████████| 1/1 [00:00<00:00,  3.27it/s]


In [6]:
len(set(originalInfected) and set(augmentedInfected)) , len(set(originalInfected) and set(randomInfected))

(396, 381)